In [1]:
!pip install -U "transformers==4.47.1" "bitsandbytes==0.45.0" "accelerate==1.2.1" "peft==0.14.0" "datasets==3.2.0" "ftfy==6.3.1" "pyarrow==18.1.0" "chardet==5.2.0" "charset-normalizer==3.3.2"

In [1]:
import os
import copy
from dataclasses import dataclass
import ftfy

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from datasets import Dataset, concatenate_datasets, Features, Value

from transformers import (
    BitsAndBytesConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

VER = '17-8BIT-R1-QWEN-MODEL-99.9PERCENT-LEFTSIDE-LMSYS-AND-33K-EXTRA-DATA-MAXLEN2200-R64-A4-BF16'

# --- lmsys extra data ---
USE_ULTRAFEEDBACK = False
USE_33K = True
USE_LMSYS = True
USE_CONCAT_DATA = True

# --- additional features ---
USE_LEFTSIDE_TRUNCATION = True

In [3]:
os.environ['WANDB_API_KEY'] = 'secret'
os.environ['WANDB_PROJECT'] = 'WSDM Qwen2.5-7b-it'
os.environ['WANDB_NOTES'] = f'WSDM Qwen2.5-7b-it LoRA Training VER-{VER}'
os.environ['WANDB_NAME'] = f'ft-qwen2.5-wsdm-ver-{VER}'

In [4]:
@dataclass
class Config:
    output_dir: str = 'output'
    checkpoint: str = 'R1-QWEN7B'
    max_length: int = 2200
    n_splits: int = 1000
    fold_idx: int = 0
    optim_type: str = 'adamw_8bit'
    per_device_train_batch_size: int = 16
    gradient_accumulation_steps: int = 1
    per_device_eval_batch_size: int = 4
    n_epochs: int = 1
    freeze_layers: int = 0
    lr: float = 2e-4
    warmup_ratio: float = 0.025
    lora_r: int = 64
    lora_alpha: float = 4.
    lora_dropout: float = 0.05
    lora_bias: str = 'none'
    num_labels: int = 2
    hdim: int = 3584
    head_dropout: float = 0.1
    label_smoothing_alpha: float = 0.
    seed: int = 0xFACED
    
config = Config()

In [5]:
training_args = TrainingArguments(
    output_dir=f'output-{VER}',
    overwrite_output_dir=True,
    report_to='wandb',
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1,
    eval_strategy='epoch',
    save_strategy='steps',
    save_steps=1000,
    optim=config.optim_type,
    bf16=True,
    learning_rate=config.lr,
    warmup_ratio=config.warmup_ratio,
    seed=config.seed,
)

In [6]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True
if USE_LEFTSIDE_TRUNCATION:
    tokenizer.padding_side = 'left'
    tokenizer.truncation_side = 'left'

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=config.num_labels,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

model.config.pad_token_id = model.config.eos_token_id

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [10]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Qwen2ForSequenceClassification(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
        

In [11]:
model.print_trainable_parameters()

trainable params: 161,487,872 || all params: 7,232,114,176 || trainable%: 2.2329


In [12]:
ds = Dataset.from_parquet('data/wsdm-original.parquet')

In [13]:
if USE_CONCAT_DATA:
    ds_concat =  Dataset.from_parquet('data/combined-lmsys-and-33k.parquet')

    ds = ds.remove_columns(['id', 'model_a', 'model_b', 'language'])
    
    ds = ds.cast(Features({
        'prompt': Value('string'),
        'response_a': Value('string'),
        'response_b': Value('string'),
        'winner': Value('string')
    }))

    ds = concatenate_datasets([ds, ds_concat])
    
    ds = ds.shuffle(seed=0xFACED)

In [14]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ['<prompt>: ' + self.process_text(t) for t in batch['prompt']]
        response_a = ['\n\n<response_a>: ' + self.process_text(t) for t in batch['response_a']]
        response_b = ['\n\n<response_b>: ' + self.process_text(t) for t in batch['response_b']]
        
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        
        labels = []
        for win in batch['winner']:
            if win in ('model_a', 'winner_a'):
                label = 0
            elif win in ('model_b', 'winner_b'):
                label = 1
            labels.append(label)
            
        return {**tokenized, 'labels': labels}

    @staticmethod
    def process_text(text: str) -> str:
        return ftfy.fix_text(text)

In [15]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

In [16]:
ds

Dataset({
    features: ['prompt', 'response_a', 'response_b', 'winner', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 102840
})

In [17]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {'acc': acc, 'log_loss': loss}

In [18]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [19]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lightsource- (lightsource-unk). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
from huggingface_hub import login, HfApi

login(token="secret")

api = HfApi()

api.upload_folder(
    folder_path=f"output-{VER}/checkpoint-6422",
    repo_id="lightsource/wsdm-qwen-r1-adapter-lmsys-and-comp-2200-int8-0.XXX-w-tta",
    repo_type="model"
)